In [1]:
!pip install transformers==4.24 cache_decorator pytorch_lightning==1.6.3 torchmetrics==0.7.0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/history')

In [4]:
import os
from event_models_utils import (
    get_rams_data_dict, load_rams_data,
)

from event_models import (
    RAMSDataModule, EventBertModel, collate_RAMS, collate_argument_RAMS
)
from transformers import (
    BertModel, BertTokenizer, BertTokenizerFast, BartTokenizer
)
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(42, workers=True)

In [ ]:
dm = RAMSDataModule(
    batch_size=8,
    num_workers=0,
    data_dir="/content/drive/MyDrive/history/datasets/rams",
    pin_memory=False
)
dm.prepare_data()
dm.setup()

In [8]:
if not os.path.exists("/content/drive/MyDrive/history/checkpoints/"):
    os.makedirs("/content/drive/MyDrive/history/checkpoints/")

In [9]:
event_checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/history/checkpoints/event_bert",
    every_n_epochs=1,
    save_top_k=2,
    monitor="valid_loss",
    mode="min"
)

In [10]:
max_epochs = 10

In [12]:
bert = BertModel.from_pretrained("bert-base-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = EventBertModel(
    bert=bert,
    tokenizer=tokenizer,
    bart_tokenizer=bart_tokenizer,
    num_events=140
)
logger = TensorBoardLogger(
    "logs", name="event_bert"
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/deprecate/deprecation.py:115: FutureWarning: The `F1` was deprecated since v0.7 in favor of `torchmetrics.classificat

In [13]:
trainer = Trainer(
    max_epochs=max_epochs,
    deterministic=True,
    accelerator="auto",
    precision=16,
    gradient_clip_val=1,
    callbacks=[event_checkpoint_callback],
    num_sanity_val_steps=0,
    logger=logger,
    #resume_from_checkpoint="/content/drive/MyDrive/history/checkpoints/event_bert/epoch=0-step=7329.ckpt"
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:51: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, dm)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:805: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  ckpt_path = ckpt_path or self.resume_from_checkpoint
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/drive/MyDrive/history/checkpoints/event_bert exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/history/checkpoints/event_bert/epoch=0-step=7329.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:1718: UserWarning: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Plea

Training: 7329it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='cuda:0')
correct:  tensor([True], device='c

Validation: 0it [00:00, ?it/s]

In [15]:
dm = RAMSDataModule(
    batch_size=8,
    num_workers=0,
    data_dir="/content/drive/MyDrive/history/datasets/rams",
    pin_memory=False
)
bert = BertModel.from_pretrained("bert-base-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = EventBertModel(
    bert=bert,
    tokenizer=tokenizer,
    bart_tokenizer=bart_tokenizer,
    num_events=140
).load_from_checkpoint(
    "/content/drive/MyDrive/history/checkpoints/event_bert/epoch=1-step=14657.ckpt",
    bert=bert,
    tokenizer=tokenizer,
    bart_tokenizer=bart_tokenizer,
    num_events=140
)
logger = TensorBoardLogger(
    "logs", name="event_bert"
)
trainer = Trainer(
    max_epochs=max_epochs,
    deterministic=True,
    accelerator="auto",
    precision=16,
    gradient_clip_val=1,
    callbacks=[event_checkpoint_callback],
    num_sanity_val_steps=0,
    logger=logger,
    #resume_from_checkpoint="/content/drive/MyDrive/history/checkpoints/event_bert/epoch=0-step=7329.ckpt"
)
trainer.test(
    model,
    dm,
    ckpt_path="/content/drive/MyDrive/history/checkpoints/event_bert/epoch=1-step=14657.ckpt"
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │   0.004592422395944595    │
│       test_f1_epoch       │   0.004592422395944595    │
│         test_loss         │     5.133366584777832     │
│      test_prec_epoch      │   0.004592422395944595    │
│     test_recall_epoch     │   0.004592422395944595    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 5.133366584777832,
  'test_acc_epoch': 0.004592422395944595,
  'test_f1_epoch': 0.004592422395944595,
  'test_prec_epoch': 0.004592422395944595,
  'test_recall_epoch': 0.004592422395944595}]